# Chapter 14: Deep Computer Vision Using Convolutional Neural Networks

This notebook contains the solution for the exercises 9, 10 and 11 of the chapter 14: *Deep Computer Vision Using Convolutional Neural Networks* of the book *Hands On Machine Learning with Scikit-Learn, Keras & TensorFlow* of Aurélien Géron.

## Configuration

In [1]:
from datetime import datetime
import os
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf 
from tensorflow.keras import (
    models,
    layers, 
    optimizers, 
    callbacks,
    activations,
    datasets, 
    applications,
    metrics
)
from tensorflow.train import Example, Features, Feature, BytesList, Int64List, FloatList

In [2]:
DATA_PATH = os.path.join('../../data', 'processed', 'mnist')
LOGS_PATH = os.path.join('../../reports', 'logs', 'mnist')
AUTOTUNE = tf.data.AUTOTUNE

# Logging configuration
logging_format = '%(message)s'
logging.basicConfig(format=logging_format, level=logging.INFO)

# Launch the TensorBoard extension
%load_ext tensorboard
%tensorboard --logdir=../../reports/logs/mnist --port=6006

## Exercise 9:

**Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST.**

### Fetch and Save the Data

In [137]:
def fetch_data():
    mnist = datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_valid = x_train[:50000], x_train[50000:]
    y_train, y_valid = y_train[:50000], y_train[50000:]
    
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    valid_set = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
    test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    
    logging.info('The dataset has been fetched.')
    return train_set, valid_set, test_set

def save_tfrecord(image, label):
    
    image = tf.io.serialize_tensor(image).numpy()
    label = label.numpy()
    
    image_example = Example(
        features = Features(
            feature = {
                'image' : Feature(bytes_list=BytesList(value=[image])),
                'label' : Feature(int64_list=Int64List(value=[label]))
    })).SerializeToString()
    
    return image_example

def save_datasets(n_files=15):
    
    train_set, valid_set, test_set = fetch_data()
    datasets = [train_set, valid_set, test_set]
    
    data_types = ['train', 'valid', 'test']
    for data_type in data_types:
        data_path = os.path.join(DATA_PATH, data_type)
        os.makedirs(data_path, exist_ok=True)

    train_paths = [os.path.join(DATA_PATH,'train', f'train_{number_file}.tfrecord') for number_file in range(n_files)]
    valid_paths = [os.path.join(DATA_PATH, 'valid', f'valid_{number_file}.tfrecord') for number_file in range(n_files)]
    test_paths = [os.path.join(DATA_PATH, 'test', f'test_{number_file}.tfrecord') for number_file in range(n_files)]
    filepaths = [train_paths, valid_paths, test_paths]
    
    for filepath, dataset in zip(filepaths, datasets):
        writers = [tf.io.TFRecordWriter(path) for path in filepath]
        for index, (image, label) in dataset.enumerate():
            n_file = index % n_files
            example = save_tfrecord(image, label)
            writers[n_file].write(example)
        logging.info(f'The {data_type} dataset has been saved as TFRecord files.')
    
    return train_paths, valid_paths, test_paths

def get_filepaths():
    
    if os.path.exists(DATA_PATH):
        data_types = ['train', 'valid', 'test']
        filepaths = []
        for data_type in data_types:
            list_files = os.listdir(os.path.join(DATA_PATH, data_type))
            filepath = [os.path.join(DATA_PATH, data_type, file) for file in list_files]
            filepaths.append(filepath)
        
        train_paths = filepaths[0]
        valid_paths = filepaths[1]
        test_paths = filepaths[2]
        
    else:
        train_paths, valid_paths, test_paths = save_datasets()
    
    logging.info('The paths for the files of the dataset have been retrieved.')
    
    return train_paths, valid_paths, test_paths

### Retrieve the Data

In [138]:
def preprocess(serialized_image):
    feature_descriptions = {
        'image' : tf.io.FixedLenFeature([], tf.string, default_value=b''),
        'label' : tf.io.FixedLenFeature([], tf.int64, default_value=0)
    }
    example = tf.io.parse_single_example(serialized_image,
                                         feature_descriptions)
    image = tf.io.parse_tensor(example['image'], out_type=tf.uint8)
    image = tf.reshape(image, shape=(28, 28, 1))
    return image, example['label']

def get_data(filepaths, shuffle_buffer_size=None, batch_size=32):
    list_files = tf.data.Dataset.list_files(filepaths)
    dataset = tf.data.TFRecordDataset(list_files, num_parallel_reads=AUTOTUNE)
    dataset = dataset.map(preprocess, num_parallel_calls=AUTOTUNE).cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    
    return dataset.batch(batch_size, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [139]:
class CustomConvLayer(layers.Layer):
    
    def __init__(self, n_filters, size_filters=3, strides=2, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.n_filters = n_filters
        self.size_filters = size_filters
        self.strides = strides
        self.activation = activations.get(activation)
        self.main_layers = [
            layers.Conv2D(n_filters, size_filters, strides=strides, padding='same', use_bias=False),
            layers.BatchNormalization(),
            self.activation,
            layers.Conv2D(n_filters, size_filters, strides=1, padding='same', use_bias=False),
            layers.BatchNormalization(),
        ]
        self.skip_layers = [
            layers.Conv2D(n_filters, 1, strides=strides, padding='same', use_bias=False),
            layers.BatchNormalization()
        ]
        
    def call(self, inputs):
        
        z = inputs
        for layer in self.main_layers:
            z = layer(z)
        skip_z = inputs
        for layer in self.skip_layers:
            skip_z = layer(skip_z)
        
        return self.activation(z + skip_z)
    
    def get_config(self):
        config =  super().get_config()
        config.update({
            'n_filters' : self.n_filters,
            'size_filters' : self.size_filters,
            'strides' : self.strides,
            'activation' : self.activation,
            'main_layers' : self.main_layers,
            'skip_layers' : self.skip_layers
        })
        return config

In [140]:
def new_model(train_set):
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(15)
    
    normalizer = layers.Normalization(input_shape=[28, 28, 1])
    sample_data = train_set.take(1000).map(lambda x, y: x)
    normalizer.adapt(sample_data)
        
    model = models.Sequential([
        normalizer,
        layers.Conv2D(64, 5, padding='same', activation='relu', input_shape=[28, 28, 1]),
        layers.BatchNormalization(),
        CustomConvLayer(128),
        CustomConvLayer(256),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(10, activation='softmax')
    ])
    
    logging.info('The model has been created.')
    
    optimizer = optimizers.Nadam()
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    logging.info('The model has been compiled.')
    
    return model


def train_model(model, train_set, valid_set, test_set):
    
    early_stopping_cb = callbacks.EarlyStopping(patience=15, restore_best_weights=True)
    tensorboard_cb = callbacks.TensorBoard(log_dir=LOGS_PATH)
    
    callbacks_list = [early_stopping_cb, tensorboard_cb]
    
    logging.info('Training the model...')
    
    model.fit(train_set,
                validation_data=valid_set,
                epochs=500,
                callbacks=callbacks_list)
    
    logging.info('The model has been trained')
    
    accuracy_model = round(model.evaluate(test_set, verbose=0)[1], 6) * 100
    logging.info(f'The accuracy of the model is {accuracy_model}%')

In [141]:
if __name__ == '__main__':
    train_filepaths, valid_filepaths, test_filepaths = get_filepaths()
    train_set = get_data(train_filepaths, shuffle_buffer_size=10000)
    valid_set = get_data(valid_filepaths)
    test_set = get_data(test_filepaths)
    
    model = new_model(train_set)
    train_model(model, train_set, valid_set, test_set)
    model.summary()

The paths for the files of the dataset have been retrieved.
2022-02-28 14:07:18.325548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-28 14:07:18.343197: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-28 14:07:20.646844: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
The model has been created.
The model has been compiled.
Training the model...


Epoch 1/500


2022-02-28 14:07:22.262180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


   1563/Unknown - 47s 29ms/step - loss: 0.2999 - accuracy: 0.9161

2022-02-28 14:08:08.991847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 51s 31ms/step - loss: 0.2999 - accuracy: 0.9161 - val_loss: 0.1046 - val_accuracy: 0.9760
Epoch 2/500
1563/1563 [==============================] - 48s 31ms/step - loss: 0.0912 - accuracy: 0.9761 - val_loss: 0.0599 - val_accuracy: 0.9841
Epoch 3/500
 723/1563 [============>.................] - ETA: 24s - loss: 0.0660 - accuracy: 0.9818

KeyboardInterrupt: 

## Exercise 11:

**Go through TensorFlow’s Style Transfer tutorial. It is a fun way to generate art
using Deep Learning.**